In [0]:
# https://keras.io/
!pip install -q keras

In [0]:
from keras.models import Sequential
from keras.callbacks import LambdaCallback, ModelCheckpoint
from keras.layers import LSTM, Dense, Activation
import itertools
import numpy as np

In [82]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
for fn, c in uploaded.items():
  with open(fn, 'wb') as f:
    f.write(c)

User uploaded file "shakespeare.txt" with length 100643 bytes


In [0]:
with open('shakespeare.txt') as f:
  lines = [line.strip(' ').lower() for line in f] #.strip('\n ,.:')
sonnets = []
ln_start = 0
ln_end = 0
for ln, content in enumerate(lines):
    if content[:-1].isdigit():
        ln_start = ln + 1
    elif not content[:-1]:
        if ln - 1 == ln_end:
            sonnets.append(lines[ln_start:ln_end + 1])
    elif ln + 1 == len(lines):
        sonnets.append(lines[ln_start:ln_end + 1])
    else:
        ln_end = ln

In [0]:
chars = sorted(set([c for s in sonnets for l in s for c in l]))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [105]:
maxlen = 40
step = 1
sentences = []
next_chars = []
text = ''.join([c for s in sonnets for l in s for c in l])
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 93587


In [106]:
chars

['\n',
 ' ',
 '!',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [0]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [0]:
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='ADAM')

In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
  
def on_epoch_end(epoch, logs):
    import random
    import sys
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [112]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
model.fit(x, y,
          batch_size=128,
          epochs=600,
          callbacks=[checkpoint])

Epoch 1/600
93587/93587 [==============================] - 52s 561us/step - loss: 0.2480

Epoch 00001: loss improved from inf to 0.24800, saving model to weights-improvement-01-0.2480.hdf5
Epoch 2/600
18816/93587 [=====>........................] - ETA: 41s - loss: 0.1550

93587/93587 [==============================] - 52s 560us/step - loss: 0.1709

Epoch 00002: loss improved from 0.24800 to 0.17092, saving model to weights-improvement-02-0.1709.hdf5
Epoch 3/600
28672/93587 [========>.....................] - ETA: 36s - loss: 0.1252

93587/93587 [==============================] - 52s 558us/step - loss: 0.1545

Epoch 00003: loss improved from 0.17092 to 0.15449, saving model to weights-improvement-03-0.1545.hdf5
Epoch 4/600
31232/93587 [=========>....................] - ETA: 34s - loss: 0.1588

93587/93587 [==============================] - 52s 558us/step - loss: 0.2089

Epoch 00004: loss did not improve
Epoch 5/600
43392/93587 [============>.................] - ETA: 27s - loss: 0.2196

93587/93587 [==============================] - 52s 556us/step - loss: 0.2449

Epoch 00005: loss did not improve
Epoch 6/600
47744/93587 [==============>...............] - ETA: 25s - loss: 0.2090

...

93587/93587 [==============================] - 53s 566us/step - loss: 0.1098

Epoch 00138: loss did not improve
Epoch 139/600
48768/93587 [==============>...............] - ETA: 25s - loss: 0.0767

93587/93587 [==============================] - 53s 562us/step - loss: 0.0907

Epoch 00139: loss did not improve
Epoch 140/600
49152/93587 [==============>...............] - ETA: 24s - loss: 0.0801

93587/93587 [==============================] - 52s 555us/step - loss: 0.0984

Epoch 00140: loss did not improve
Epoch 141/600
49280/93587 [==============>...............] - ETA: 24s - loss: 0.1087

93587/93587 [==============================] - 52s 552us/step - loss: 0.1151

Epoch 00141: loss did not improve
Epoch 142/600
33792/93587 [=========>....................] - ETA: 32s - loss: 0.0824

KeyboardInterrupt: ignored

In [113]:
!ls

datalab				     weights-improvement-26-0.5547.hdf5
example.txt			     weights-improvement-27-0.5253.hdf5
first				     weights-improvement-28-0.5036.hdf5
shakespeare.txt			     weights-improvement-29-0.4801.hdf5
weights-improvement-01-0.1083.hdf5   weights-improvement-30-0.4562.hdf5
weights-improvement-01-0.2480.hdf5   weights-improvement-31-0.4357.hdf5
weights-improvement-01-2.4741.hdf5   weights-improvement-32-0.4259.hdf5
weights-improvement-02-0.0610.hdf5   weights-improvement-33-0.4045.hdf5
weights-improvement-02-0.1709.hdf5   weights-improvement-34-0.3924.hdf5
weights-improvement-02-2.0126.hdf5   weights-improvement-35-0.1495.hdf5
weights-improvement-03-0.0344.hdf5   weights-improvement-35-0.3824.hdf5
weights-improvement-03-0.1545.hdf5   weights-improvement-36-0.3712.hdf5
weights-improvement-03-1.8378.hdf5   weights-improvement-37-0.3560.hdf5
weights-improvement-04-0.0191.hdf5   weights-improvement-38-0.3478.hdf5
weights-improvement-04-1.7308.hdf5   weights-improvement-39-0.1413.h

In [0]:
from google.colab import files

files.download('weights-improvement-135-0.0870.hdf5')

In [121]:
import random
import sys
start_index = random.randint(0, len(text) - maxlen - 1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
    print('----- diversity:', diversity)

    generated = ''
    sentence = 'thy glass will show thee how thy beautie'#text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

----- diversity: 0.2
----- Generating with seed: "thy glass will show thee how thy beautie"
thy glass will show thee how thy beauties wear,
thy dial how thy precious minutes waste,
these vacant leaves thy mind's imprint will bear,
and of this book, this learning mads of slow,
to sincuse to the earthanct of my love,
thy self thy self thy e-didelich praise.
like as the wadds but of things to rise mine eye,
and all my soul, and all my every part;
and for this sin 

the child of him and kee)
leath trively for a kind of beary canse,

----- diversity: 0.5
----- Generating with seed: "thy glass will show thee how thy beautie"
thy glass will show thee how thy beauties wear,
thy dial how thy precious minutes waste,
these vacant leaves thy mind's imprint will bear,
and make the larged bud your monnory,
or if they cruelioned when i spence.
but sweets in thy fair perpessed that died.
but wherefore do die, my self alone hearts
that love is as your bo

unty doth deceive.
and that thou tencels in all you a wood.
i heavy seear thou art full-fleater,
i sable grace varin
----- diversity: 1.0
----- Generating with seed: "thy glass will show thee how thy beautie"
thy glass will show thee how thy beauties wear,
thy dial how thy precious minutes waste,
but now to gone, for another, shap was tomb.
thy beauty styen's are part in warth grows
loads woild have toon might in yet the share,
and thy strangely gave no more be cheal,
to let not when you for 

him in ithan your sweet,
finding that buding have i fasted time.
or may the eye hath flething, and i seem,
till now all-ummant with her of a spory,
sama
----- diversity: 1.2
----- Generating with seed: "thy glass will show thee how thy beautie"
thy glass will show thee how thy beauties wear,
thy dial hy admeate, art to make that burse,
but yet thou mayst in me thou mayst can see,
so now i have your longer' bads ang ermed,
and to thy please him what i thy pars, feelfurned,
or ance fine my love and thing

s of lears,
thrin shows my mistress vexceled, and i brevided
whon'ersmorn pergeauty not to his, and in
mine works wenk of your disgrace, and cheeks new rade spirith:
and such a c
